# Running on AWS

*Author: Creare* <br>
*Date: April 01 2020* <br>

**Keywords**: processing, cloud, AWS

## Overview

Runing PODPAC Pipeline on AWS Lambda.

### Prerequisites

- Python 2.7 or above
- [`podpac[datatype]`](https://podpac.org/install.html#install)
- [`podpac[aws]`](https://podpac.org/install.html#install)
- *Review the [README.md](../../README.md) and [jupyter-tutorial.ipynb](../jupyter-tutorial.ipynb) for additional info on using jupyter notebooks*


### See Also

- [AWS Lambda Tutorial](../4-advanced/aws-lambda.ipynb): Tutorial for **setting up the PODPAC Lambda function on AWS**
- [AWS Budgeting Tutorial](../4-advanced/aws-budget.ipynb): Tutorial for budgeting with PODPAC and AWS
- [SMAP Datalib](../5-datalib/smap/010-retrieving-SMAP-data.ipynb): SMAP Datalib tutorial
- [100-analyzing-SMAP-data.ipynb](../5-datalib/smap/100-analyzing-SMAP-data.ipynb)

# How to run analysis on AWS Lambda

<i style='font-size:18pt'>PODPAC uses the AWS commandline tools to trigger the Lambda function execution. </i>

<img src='../../images/PODPAC-AWS-Lambda.png' style='width:80%; margin-left:auto;margin-right:auto' />

We will:
* Create a node
* Execute the node on AWS Lambda

**This example requires a pre-configured AWS Lambda node.** We will not cover how the AWS Lambda node was created. 

In [1]:
%matplotlib inline
# Get the PODPAC logger
import logging
logger = logging.getLogger("podpac")

# Setup PODPAC for AWS (1/2)

* Configure AWS credentials
* Can be specified at runtime, or in the podpac settings file

In [2]:
import podpac
from podpac import settings

# need to allow unsafe evaluation to use `podpac.algorithm.Arithmetic` Node (below) 
settings.set_unsafe_eval(True)

# # Credentials
# settings["AWS_ACCESS_KEY_ID"] = "access key id"
# settings["AWS_SECRET_ACCESS_KEY"] = "secrect access key"
# settings["AWS_REGION_NAME"] = "region name"

# # General Settings
# settings["AWS_TAGS"] = {} # tags to assign to AWS resources created through PODPAC

# # S3
# settings["S3_BUCKET_NAME"] = "S3 bucket for Lambda functions or PODPAC cache"

# # Lambda
# settings["FUNCTION_NAME"] = "name of lambda function to eval"
# settings["FUNCTION_ROLE_NAME"] = "role name for lambda function"
# settings["FUNCTION_DEPENDENCIES_KEY"] = "path on S3 bucket where function depedencies live"
# settings["FUNCTION_S3_INPUT"] = "path on S3 bucket for input pipelines. Objects put in this directory will trigger lambda function",
# settings["FUNCTION_S3_OUTPUT"] = "path on S3 bucket for pipeline outputs. Objects put in this directory will be returned to lambda function",

# # Paths - overwrite paths for Lambda caching
# this will be fixed in future releases of PODPAC
settings["ROOT_PATH"] = "/tmp/"
settings["LOG_FILE_PATH"] = "/tmp/"

## Provide Earth Data Login Credentials
If you do not have an earth data login, or have not activated OpenDAP access, follow the [instructions here](https://creare-com.github.io/podpac-docs/user/earthdata.html).

In [3]:
import getpass
username = password = None
username = input("Username:");   password = getpass.getpass('Password:')

# EarthData Credentials need to get saved in `settings` in order to get passed
# into the Lambda function on AWS. This will be automated in the future.
settings["username@urs.earthdata.nasa.gov"] = username
settings["password@urs.earthdata.nasa.gov"] = password

Username: mpuecker
Password: ··················


# Setup (2/2)

* Create the PODPAC Pipeline
* We'll use the same pipeline from the [100-analyzing-SMAP-data.ipynb](../5-datalib/smap/100-analyzing-SMAP-data.ipynb) notebook
* This example computes the difference between the current soil moisture for a region, and that of the previous year

In [4]:
import podpac.datalib

# Create the Pipsmap_offset
product = 'SPL4SMAU'
smap = podpac.datalib.smap_egi.SMAP(product=product)
smap_time1_offset = podpac.algorithm.ExpandCoordinates(source=smap, time=['-1,Y', '-1,Y', '1,Y'])
smap_offset = podpac.algorithm.Mean(source=smap_time1_offset, dims=['time'])

# This is the output Node of the Pipeline
diff = podpac.algorithm.Arithmetic(eqn='B-A', A=smap, B=smap_offset)

# Create PODPAC Coordinates 
* This specifies the region and date where the pipeline will be evaluated

In [5]:
# Specify region of interest on a uniform grid
lat = podpac.crange(  60,  10, -2.0)  # (start, stop, step)
lon = podpac.crange(-130, -60,  2.0)  # (start, stop, step)

# Specify date and time
time = '2018-05-19T12:00:00'

# Create the PODPAC Coordinates
coords = podpac.Coordinates([lat, lon, time], dims=['lat', 'lon', 'time'])

# Evaluating node on AWS cloud

In [6]:
# Set up wrapper to run your node on AWS Lambda
node = podpac.managers.aws.Lambda(source=diff, function_allow_unsafe_eval=True)

In [7]:
# if this is the first time your are using this node, build the AWS resources
# if you have used this function before, skip this step
node.build()

In [8]:
# validate that the AWS resources are built
node.validate()

# review aws resources
node.describe()


Lambda Node (built)
    Function
        Name: podpac-test-lambda-notebook
        Description: PODPAC Lambda Function (https://podpac.org)
        ARN: arn:aws:lambda:us-east-1:120507435541:function:podpac-test-lambda-notebook
        Triggers: ['eval']
        Handler: handler.handler
        Environment Variables: {'PODPAC_UNSAFE_EVAL': 'b754a7523c574ec0b88c9c831a4e5ffb'}
        Timeout: 600 seconds
        Memory: 2048 MB
        Tags: {'_podpac_resource': 'true', '_podpac_resource_hash': 'c9608f37a7fb154b895274f4e050c8e6', 'acct': '1010115.01.003', 'owner': 'mpu'}
        Source Dist: eb2vQrp3ocuODfenDbzRJwJyAHoG/Eem+rt3cgN4+Dg=
        Source Dependencies: s3://podpac-test-lambda-notebook/podpac_deps.zip
        Last Modified: 2020-04-22T19:17:19.223+0000
        Version: $LATEST
        Restrict Evaluation: []

    S3
        Bucket: podpac-test-lambda-notebook
        Tags: {'owner': 'mpu', '_podpac_resource': 'true', '_podpac_resource_hash': 'c9608f37a7fb154b895274f4e050c8e6

In [ ]:
# Run the node (on AWS Lambda) and get the results back
# (the first time this runs will be slower than subsequent calls)
output = node.eval(coords)
output.plot()
pass

In [ ]:
o = podpac.Node.from_json(diff.json).eval(podpac.Coordinates.from_json(coords.json))
o.plot()